In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime as dt

In [2]:
# Upload csv

stock_data_csv = "C:/Users/Satinder Bains/Desktop/Stocks Vs Gold Allocation/Datasets/snp500.csv"
stock_data_csv

gold_data_csv = "C:/Users/Satinder Bains/Desktop/Stocks Vs Gold Allocation/Datasets/goldx.csv"
gold_data_csv

gold_extended = "C:/Users/Satinder Bains/Desktop/Stocks Vs Gold Allocation/Datasets/Gold-quandl.csv"
gold_extended

'C:/Users/Satinder Bains/Desktop/Stocks Vs Gold Allocation/Datasets/Gold-quandl.csv'

In [3]:
# The correct encoding must be used to read the CSV in pandas
gold_df = pd.read_csv(gold_data_csv, encoding='utf-8')
gold_df.head()

,Date,Price,Open,High,Low
0,"Aug 01, 2018",1216.6,1223.4,1223.4,1216.2
1,"Jul 31, 2018",1223.7,1220.4,1228.1,1213.0
2,"Jul 30, 2018",1221.3,1222.5,1223.9,1218.1
3,"Jul 27, 2018",1222.2,1223.5,1226.8,1216.7
4,"Jul 26, 2018",1225.3,1227.8,1227.8,1227.8


### Cleaning Stock Data

In [4]:
# The correct encoding must be used to read the CSV in pandas
stocks_df = pd.read_csv(stock_data_csv, encoding='utf-8')
stocks_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


In [5]:
# Check Datatypes
stocks_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [6]:
# Count Columns
stocks_df.count()

Date         17543
Open         17543
High         17543
Low          17543
Close        17543
Adj Close    17543
Volume       17543
dtype: int64

In [7]:
# Delete extraneous column
del stocks_df['Volume']
del stocks_df['Adj Close']
stocks_df.head()

,Date,Open,High,Low,Close
0,1950-01-03,16.66,16.66,16.66,16.66
1,1950-01-04,16.85,16.85,16.85,16.85
2,1950-01-05,16.93,16.93,16.93,16.93
3,1950-01-06,16.98,16.98,16.98,16.98
4,1950-01-09,17.08,17.08,17.08,17.08


In [8]:
# Find the Date
# Loc and Iloc also allow for conditional statments to filter rows of data
# using Loc on the logic test above only returns rows where the result is True
only_billys = stocks_df.loc[stocks_df["Date"] == "1980-01-01", :]
print(only_billys)

Empty DataFrame
Columns: [Date, Open, High, Low, Close]
Index: []


In [9]:
#FormatDatesProperly
newFormat = pd.to_datetime(stocks_df['Date'])

In [10]:
dataformat = stocks_df.set_index('Date')
dataformat.head()

,Open,High,Low,Close
Date,,,,
1950-01-03,16.66,16.66,16.66,16.66
1950-01-04,16.85,16.85,16.85,16.85
1950-01-05,16.93,16.93,16.93,16.93
1950-01-06,16.98,16.98,16.98,16.98
1950-01-09,17.08,17.08,17.08,17.08


In [11]:
# Delete extraneous column
del stocks_df['Date']
stocks_df.head()

,Open,High,Low,Close
0,16.66,16.66,16.66,16.66
1,16.85,16.85,16.85,16.85
2,16.93,16.93,16.93,16.93
3,16.98,16.98,16.98,16.98
4,17.08,17.08,17.08,17.08


In [12]:
#Concat datasets
stocks_df = pd.concat([stocks_df, newFormat], axis=1)
stocks_df.head()

,Open,High,Low,Close,Date
0,16.66,16.66,16.66,16.66,1950-01-03
1,16.85,16.85,16.85,16.85,1950-01-04
2,16.93,16.93,16.93,16.93,1950-01-05
3,16.98,16.98,16.98,16.98,1950-01-06
4,17.08,17.08,17.08,17.08,1950-01-09


In [13]:
dataformat = stocks_df.set_index('Date')
dataformat.head()

,Open,High,Low,Close
Date,,,,
1950-01-03,16.66,16.66,16.66,16.66
1950-01-04,16.85,16.85,16.85,16.85
1950-01-05,16.93,16.93,16.93,16.93
1950-01-06,16.98,16.98,16.98,16.98
1950-01-09,17.08,17.08,17.08,17.08


# Gold Cleaning

## Part I

In [14]:
# The correct encoding must be used to read the CSV in pandas
gold_extend = pd.read_csv(gold_extended, encoding='utf-8')
gold_extend.head()

,Date,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
0,2017-10-24,1278.30,1276.45,970.36,970.48,1087.32,1085.62
1,2017-10-23,1275.25,1274.90,967.79,966.37,1085.62,1085.07
2,2017-10-20,1280.25,1281.20,974.27,973.20,1084.76,1086.97
3,2017-10-19,1283.40,1286.40,975.64,975.35,1087.42,1085.98
4,2017-10-18,1280.65,1280.20,972.53,973.08,1090.47,1088.55


In [15]:
#Reverse order so lowest dates show first
aux = gold_extend.iloc[::-1]
aux.head()

,Date,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
12590,1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN
12589,1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN
12588,1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN
12587,1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN
12586,1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN


In [16]:
aux.fillna

<bound method DataFrame.fillna of              Date  USD (AM)  USD (PM)  GBP (AM)  GBP (PM)  EURO (AM)  \
12590  1968-01-02     35.18       NaN    14.641       NaN        NaN   
12589  1968-01-03     35.16       NaN    14.617       NaN        NaN   
12588  1968-01-04     35.14       NaN    14.603       NaN        NaN   
12587  1968-01-05     35.14       NaN    14.597       NaN        NaN   
12586  1968-01-08     35.14       NaN    14.586       NaN        NaN   
12585  1968-01-09     35.14       NaN    14.576       NaN        NaN   
12584  1968-01-10     35.15       NaN    14.576       NaN        NaN   
12583  1968-01-11     35.17       NaN    14.596       NaN        NaN   
12582  1968-01-12     35.18       NaN    14.607       NaN        NaN   
12581  1968-01-15     35.18       NaN    14.597       NaN        NaN   
12580  1968-01-16     35.19       NaN    14.596       NaN        NaN   
12579  1968-01-17     35.20       NaN    14.613       NaN        NaN   
12578  1968-01-18     35.20   

In [17]:
#FormatDatesProperly
auDate = pd.to_datetime(aux['Date'])
auDate.head()

12590   1968-01-02
12589   1968-01-03
12588   1968-01-04
12587   1968-01-05
12586   1968-01-08
Name: Date, dtype: datetime64[ns]

In [18]:
#Delete Object "Date"
del aux['Date']

In [19]:
# Add Date Format to Dataframe
#if you want to specify the order of the column, you can use insert
#here, we are inserting at index 1 (so should be second col in dataframe)
#here is the simplist way to add the new column
auDate  = pd.concat([aux, auDate], axis=1)
auDate.head()

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM),Date
12590,35.18,NaN,14.641,NaN,NaN,NaN,1968-01-02
12589,35.16,NaN,14.617,NaN,NaN,NaN,1968-01-03
12588,35.14,NaN,14.603,NaN,NaN,NaN,1968-01-04
12587,35.14,NaN,14.597,NaN,NaN,NaN,1968-01-05
12586,35.14,NaN,14.586,NaN,NaN,NaN,1968-01-08


In [20]:
auDate.dtypes

USD (AM)            float64
USD (PM)            float64
GBP (AM)            float64
GBP (PM)            float64
EURO (AM)           float64
EURO (PM)           float64
Date         datetime64[ns]
dtype: object

In [21]:
indexAU = auDate.set_index('Date')
indexAU.head()

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
Date,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN


## Part II

In [22]:
gold_df.head()

,Date,Price,Open,High,Low
0,"Aug 01, 2018",1216.6,1223.4,1223.4,1216.2
1,"Jul 31, 2018",1223.7,1220.4,1228.1,1213.0
2,"Jul 30, 2018",1221.3,1222.5,1223.9,1218.1
3,"Jul 27, 2018",1222.2,1223.5,1226.8,1216.7
4,"Jul 26, 2018",1225.3,1227.8,1227.8,1227.8


In [23]:
# Check Datatype
gold_df.dtypes

Date      object
Price    float64
Open     float64
High     float64
Low      float64
dtype: object

In [24]:
#FormatDatesProperly
newFormatGold = pd.to_datetime(gold_df['Date'])
newFormatGold.head()

0   2018-08-01
1   2018-07-31
2   2018-07-30
3   2018-07-27
4   2018-07-26
Name: Date, dtype: datetime64[ns]

In [25]:
newFormatGold.dtype

dtype('<M8[ns]')

In [26]:
del gold_df['Date']

In [27]:
# Add Date Format to Dataframe
#if you want to specify the order of the column, you can use insert
#here, we are inserting at index 1 (so should be second col in dataframe)
#here is the simplist way to add the new column
au_ya  = pd.concat([gold_df, newFormatGold], axis=1)
au_ya.head()

,Price,Open,High,Low,Date
0,1216.6,1223.4,1223.4,1216.2,2018-08-01
1,1223.7,1220.4,1228.1,1213.0,2018-07-31
2,1221.3,1222.5,1223.9,1218.1,2018-07-30
3,1222.2,1223.5,1226.8,1216.7,2018-07-27
4,1225.3,1227.8,1227.8,1227.8,2018-07-26


In [28]:
#Reverse order so lowest dates show first
au_ya = au_ya.iloc[::-1]
au_ya.head()

,Price,Open,High,Low,Date
9770,515.5,517.00,517.0,513.00,1979-12-27
9769,517.8,516.00,517.8,510.40,1979-12-28
9768,533.6,527.88,534.5,527.88,1979-12-31
9767,575.5,562.50,577.0,558.00,1980-01-02
9766,625.0,627.00,640.0,603.00,1980-01-03


In [29]:
au_ya.dtypes

Price           float64
Open            float64
High            float64
Low             float64
Date     datetime64[ns]
dtype: object

In [30]:
gold_final = au_ya.set_index('Date')
gold_final.head()

,Price,Open,High,Low
Date,,,,
1979-12-27,515.5,517.00,517.0,513.00
1979-12-28,517.8,516.00,517.8,510.40
1979-12-31,533.6,527.88,534.5,527.88
1980-01-02,575.5,562.50,577.0,558.00
1980-01-03,625.0,627.00,640.0,603.00


# MERGEDASET

In [31]:
#Merge Gold Dataset
mergeAU = pd.merge(indexAU, gold_final, on="Date", how="outer")
mergeAU.head()

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM),Price,Open,High,Low
Date,,,,,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#Merge datasets
merge_table = pd.merge(mergeAU, dataformat, on="Date", how='outer')
merge_table.head()

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM),Price,Open_x,High_x,Low_x,Open_y,High_y,Low_y,Close
Date,,,,,,,,,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003


In [33]:
# Using .rename(columns={}) in order to rename columns
renamed_df = merge_table.rename(columns={"High_y":"SP.High", "Low_y":"SP.Low", "Price":"AU.Price",
                                        "Open_x":"AU.Open", "High_x":"AU.High", "Low_x":"AU.Low", 'Close':'SP.Price',
                                        'Open_y':'SP.Open', 'USD (AM)':'Gold.Price', 'USD (PM)':'Gold.Close'})
renamed_df.head()

,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price
Date,,,,,,,,,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003


In [34]:
#Fill price of Gold.Price with Au.Price updated to end of 79
test1 = renamed_df['AU.Price'].fillna(renamed_df['Gold.Price'])
test1.head()

Date
1968-01-02    35.18
1968-01-03    35.16
1968-01-04    35.14
1968-01-05    35.14
1968-01-08    35.14
Name: AU.Price, dtype: float64

In [35]:
#Fill price of Gold.Price with Au.Price updated to end of 79
test2 = renamed_df['Gold.Price'].fillna(renamed_df['AU.Price'])
test2.head()

Date
1968-01-02    35.18
1968-01-03    35.16
1968-01-04    35.14
1968-01-05    35.14
1968-01-08    35.14
Name: Gold.Price, dtype: float64

In [36]:
renamed_df['AU.Price'] = renamed_df['AU.Price'].fillna(value=renamed_df['Gold.Price'])
renamed_df.head(3050)

,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price
Date,,,,,,,,,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003
1968-01-09,35.14,NaN,14.576,NaN,NaN,NaN,35.14,NaN,NaN,NaN,96.620003,97.839996,95.889999,96.500000
1968-01-10,35.15,NaN,14.576,NaN,NaN,NaN,35.15,NaN,NaN,NaN,96.500000,97.260002,95.660004,96.519997
1968-01-11,35.17,NaN,14.596,NaN,NaN,NaN,35.17,NaN,NaN,NaN,96.519997,97.820000,95.879997,96.620003
1968-01-12,35.18,NaN,14.607,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.620003,97.440002,95.870003,96.720001


In [37]:
final = renamed_df.reset_index()
final.head()

,Date,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price
0,1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001
1,1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998
2,1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001
3,1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002
4,1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003


In [38]:
final.count()

Date          17860
Gold.Price    12590
Gold.Close    12456
GBP (AM)      12580
GBP (PM)      12445
EURO (AM)      4754
EURO (PM)      4719
AU.Price      12988
AU.Open        9771
AU.High        9771
AU.Low         9771
SP.Open       17543
SP.High       17543
SP.Low        17543
SP.Price      17543
dtype: int64

In [39]:
#final['Date1'] = final['Date'].dt.strftime('%m/%d/%Y')
#print (final)

final['DATE'] = final['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
final.head()

,Date,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price,DATE
0,1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001,1968-01-02
1,1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998,1968-01-03
2,1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001,1968-01-04
3,1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002,1968-01-05
4,1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003,1968-01-08


In [40]:
#final['Date1'] = final['Date'].dt.strftime('%m/%d/%Y')
#final.head()

In [41]:
date_format = pd.to_datetime(final['DATE'])
date_format.head()

0   1968-01-02
1   1968-01-03
2   1968-01-04
3   1968-01-05
4   1968-01-08
Name: DATE, dtype: datetime64[ns]

In [42]:
del final['DATE']
#del final['Date']

In [43]:
final_format  = pd.concat([final, date_format], axis=1)
final_format.head()

,Date,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price,DATE
0,1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001,1968-01-02
1,1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998,1968-01-03
2,1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001,1968-01-04
3,1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002,1968-01-05
4,1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003,1968-01-08


In [44]:
final_format.dtypes

Date          datetime64[ns]
Gold.Price           float64
Gold.Close           float64
GBP (AM)             float64
GBP (PM)             float64
EURO (AM)            float64
EURO (PM)            float64
AU.Price             float64
AU.Open              float64
AU.High              float64
AU.Low               float64
SP.Open              float64
SP.High              float64
SP.Low               float64
SP.Price             float64
DATE          datetime64[ns]
dtype: object

In [45]:
final_format['Year'] = final_format['DATE'].dt.year

final_format['Month'] = final_format['DATE'].dt.month

final_format['Day'] = final_format['DATE'].dt.day

In [46]:
# Remove two columns name is 'C' and 'D' 
final_format = final_format.drop(['DATE'], axis = 1)

In [47]:
practiceloc = final_format.set_index('Date')
practiceloc.head()

,Gold.Price,Gold.Close,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price,Year,Month,Day
Date,,,,,,,,,,,,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001,1968,1,2
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998,1968,1,3
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001,1968,1,4
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002,1968,1,5
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003,1968,1,8


In [48]:
#delete extraneous columns
droptop = practiceloc.drop(['Gold.Price', 'Gold.Close'], axis = 1)
droptop.head()

,GBP (AM),GBP (PM),EURO (AM),EURO (PM),AU.Price,AU.Open,AU.High,AU.Low,SP.Open,SP.High,SP.Low,SP.Price,Year,Month,Day
Date,,,,,,,,,,,,,,,
1968-01-02,14.641,NaN,NaN,NaN,35.18,NaN,NaN,NaN,96.470001,97.330002,95.309998,96.110001,1968,1,2
1968-01-03,14.617,NaN,NaN,NaN,35.16,NaN,NaN,NaN,96.110001,96.949997,95.040001,95.669998,1968,1,3
1968-01-04,14.603,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.669998,96.230003,94.309998,95.360001,1968,1,4
1968-01-05,14.597,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.360001,96.660004,94.970001,95.940002,1968,1,5
1968-01-08,14.586,NaN,NaN,NaN,35.14,NaN,NaN,NaN,95.940002,97.400002,95.540001,96.620003,1968,1,8


In [49]:
droptop = droptop[[
    'SP.Price','SP.Open','SP.Low','SP.High',
    'AU.Price','AU.Open','AU.Low','AU.High',
    'GBP (AM)','GBP (PM)','EURO (AM)','EURO (PM)',
    'Month','Day','Year']]
droptop.head()

,SP.Price,SP.Open,SP.Low,SP.High,AU.Price,AU.Open,AU.Low,AU.High,GBP (AM),GBP (PM),EURO (AM),EURO (PM),Month,Day,Year
Date,,,,,,,,,,,,,,,
1968-01-02,96.110001,96.470001,95.309998,97.330002,35.18,NaN,NaN,NaN,14.641,NaN,NaN,NaN,1,2,1968
1968-01-03,95.669998,96.110001,95.040001,96.949997,35.16,NaN,NaN,NaN,14.617,NaN,NaN,NaN,1,3,1968
1968-01-04,95.360001,95.669998,94.309998,96.230003,35.14,NaN,NaN,NaN,14.603,NaN,NaN,NaN,1,4,1968
1968-01-05,95.940002,95.360001,94.970001,96.660004,35.14,NaN,NaN,NaN,14.597,NaN,NaN,NaN,1,5,1968
1968-01-08,96.620003,95.940002,95.540001,97.400002,35.14,NaN,NaN,NaN,14.586,NaN,NaN,NaN,1,8,1968


In [50]:
UncleSam = droptop[[
    'SP.Price','SP.Open','SP.Low','SP.High',
    'AU.Price','AU.Open','AU.Low','AU.High',
    'Month','Day','Year']]
UncleSam.head()

,SP.Price,SP.Open,SP.Low,SP.High,AU.Price,AU.Open,AU.Low,AU.High,Month,Day,Year
Date,,,,,,,,,,,
1968-01-02,96.110001,96.470001,95.309998,97.330002,35.18,NaN,NaN,NaN,1,2,1968
1968-01-03,95.669998,96.110001,95.040001,96.949997,35.16,NaN,NaN,NaN,1,3,1968
1968-01-04,95.360001,95.669998,94.309998,96.230003,35.14,NaN,NaN,NaN,1,4,1968
1968-01-05,95.940002,95.360001,94.970001,96.660004,35.14,NaN,NaN,NaN,1,5,1968
1968-01-08,96.620003,95.940002,95.540001,97.400002,35.14,NaN,NaN,NaN,1,8,1968


In [51]:
# Grab the data contained within the "Berry" row and the "Phone Number" column
#simran = practiceloc.loc["1987-08-17", "AU.Price"]
#print("Using Loc: " + simran)

In [52]:
#Store time interval periods in dictionary
#one_yr = []
#two_yr = []
#five_yr = []
#ten_yr = []

#years = final_format["Year"]
#months = final_format['Month']
#days = final_format['Day']
#gold_price = final_format['AU.Price']
#sp_price = final_format['SP.Close']

#Loop to grab data between time intervals and store them

#def my():
#    for year in final_format:
#        if(year==1979):
#            print(year)
#        if(year!=1979):
#            pass

In [53]:
close = final_format[['Year', 'AU.Price', 'SP.Close']]
close.head()

KeyError: "['SP.Close'] not in index"

In [ ]:
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2012, 8, 23) - dt.timedelta(days=365)